In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import *

%matplotlib inline

# [Inserte nombre de proyecto]

- Jorge Andres Mogotocoro Fajardo - 2160075
- 
- 

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#url example './drive/My Drive/Colab Notebooks/Python/Cursos/2019-1 Inteligencia Artificial I/'
# ./drive/My Drive/Colab Notebooks/Python/Cursos/2019-1 Inteligencia Artificial I/Proyectos/Resumen de las tasas de suicidio de 1985 a 2016/master.csv

**No olvidar poner en la variable path la direccion de drive donde se trabaja...** \\
**NOTA:** Experimentando un poco he llegado a la conclusion que no es tan eficiente trabajar varios en un mismos documento por tanto sugiero lo siguiente:

- Las variables *path* dado que es dependiente de cada uno por favor usar celdas independiente con su propia direccion asi el proximo que llegue a editar puede volver a ejecutar su progreso anterior.
- No editar las celdas ya implementadas, en lo posible mejor agregar comentarios sobre dudas o errores cometidos.

#### Contenido:
Este conjunto de datos compilado se extrajo de otros cuatro conjuntos de datos vinculados por tiempo y lugar, y se creó para encontrar señales correlacionadas con el aumento de las tasas de suicidio entre diferentes cohortes a nivel mundial, en todo el espectro socioeconómico.

#### Columnas:
país, año, sexo, grupo de edad, conteo de suicidios, población, tasa de suicidios, clave compuesta país-año, IDH por año, gdp_for_year, gdp_per_capita, generación (según el promedio de agrupación por edad).

#### Definiciones:

- **HDI (IDH):** Indice de desarrollo humano. Es un indicador sintético de los logros,medios obtenidos en las dimensiones fundamentales del desarrollo humano, a saber, tener una vida larga y saludable, adquirir conocimientos y disfrutar de un nivel de vida digno.
- **GDP (PIB):** Producto interno bruto. En unidades de (Dólares estadounidenses actuales). Las fuentes incluyen: Banco Mundial, Naciones Unidas.

## Objetivos (borrador):

#### Clasificacion (**clases bien definidas, pertenecen a los naturales**):

#### Regresion (**Pertenecen a los reales**):

- Predecir el numero de suicidos (suicides_no) para unos determinados datos de entrada (country, year, sex, age, ¿population?).
- Precedir la probabilidad de que una persona se suicide (**suicides_no(desconocido) / population**) segun ciertas condiciones (**Locacion especifica, condiciones de vida, ...**) <p> **Posibles complicaciones**: Solo se estaria determinando la probabilidad a partir de las condiciones de vida sociales dejando de lado las personales, ¿hasta que punto es posible realizar regrecion? teniendo en cuenta la caractaristica **year**.</p>
- Determinar patrones de acuerdo a ciertas caracteristicas que presenten mayor influencia en las tazas de suicidios.

## Cargar datos:

In [0]:
# No olvidar poner en la variable path la direccion de drive donde se trabaja...
path = './drive/My Drive/Colab Notebooks/Python/Cursos/2019-1 Inteligencia Artificial I/Proyectos/Resumen de las tasas de suicidio de 1985 a 2016/' 

In [5]:
d = pd.read_csv(path+'master.csv')
country_codes = pd.read_csv(path+"wikipedia-iso-country-codes.csv")
print d.shape

(27820, 12)


In [24]:
d.head()

,country,year,sex,age,suicides_no,population,suicides/100k_pop,country-year,HDI_for_year,gdp_for_year,gdp_per_capita,generation
0,8,1987,0,1,21,312900,6.71,Albania1987,NaN,2.157,796,3
1,8,1987,0,3,16,308000,5.19,Albania1987,NaN,2.157,796,1
2,8,1987,1,1,14,289700,4.83,Albania1987,NaN,2.157,796,3
3,8,1987,0,5,1,21800,4.59,Albania1987,NaN,2.157,796,0
4,8,1987,0,2,9,274300,3.28,Albania1987,NaN,2.157,796,2


In [25]:
d.tail()

,country,year,sex,age,suicides_no,population,suicides/100k_pop,country-year,HDI_for_year,gdp_for_year,gdp_per_capita,generation
27815,860,2014,1,3,107,3620833,2.96,Uzbekistan2014,0.675,63.067,2309,3
27816,860,2014,1,5,9,348465,2.58,Uzbekistan2014,0.675,63.067,2309,1
27817,860,2014,0,0,60,2762158,2.17,Uzbekistan2014,0.675,63.067,2309,5
27818,860,2014,1,0,44,2631600,1.67,Uzbekistan2014,0.675,63.067,2309,5
27819,860,2014,1,4,21,1438935,1.46,Uzbekistan2014,0.675,63.067,2309,2


In [0]:
set(d.values[:,3])
#set(d.values[:,3])
#set(d.values[:,0])
#set(d.values[:,1])

{'15-24 years',
 '25-34 years',
 '35-54 years',
 '5-14 years',
 '55-74 years',
 '75+ years'}

In [0]:
for i in range(d.shape[0]):
    print(d.values[i,7] == d.values[i,0]+str(d.values[i,1]))

 **Conclusion:** Las columnas "country", "year" y "country-year" son basicamente la misma informacion

Eleccion de caracteristicas:
<p> X { <b>country (string) ,year ,sex ,age (string),suicides_no ,population ,HDI for year ,gdp_for_year ,gdp_per_capita ,generation (string)} </b></p>
<p> y { <b> suicides_no </b> } </p>
<p> <b>Posibles complicaciones</b>: <b>country</b> debe pasarse a valores de cordenadas o una representacion numerica, <b>sex</b> podria tomar valores de 0 y 1, ¿como deberia manejarse <b>generacion</b>?</p>

### Country
Albania',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Bosnia and Herzegovina',
 'Brazil',
 'Bulgaria',
 'Cabo Verde',
 'Canada',
 'Chile',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Denmark',
 'Dominica',
 'Ecuador',
 'El Salvador',
 'Estonia',
 'Fiji',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guyana',
 'Hungary',
 'Iceland',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Kazakhstan',
 'Kiribati',
 'Kuwait',
 'Kyrgyzstan',
 'Latvia',
 'Lithuania',
 'Luxembourg',
 'Macau',
 'Maldives',
 'Malta',
 'Mauritius',
 'Mexico',
 'Mongolia',
 'Montenegro',
 'Netherlands',
 'New Zealand',
 'Nicaragua',
 'Norway',
 'Oman',
 'Panama',
 'Paraguay',
 'Philippines',
 'Poland',
 'Portugal',
 'Puerto Rico',
 'Qatar',
 'Republic of Korea',
 'Romania',
 'Russian Federation',
 'Saint Kitts and Nevis',
 'Saint Lucia',
 'Saint Vincent and Grenadines',
 'San Marino',
 'Serbia',
 'Seychelles',
 'Singapore',
 'Slovakia',
 'Slovenia',
 'South Africa',
 'Spain',
 'Sri Lanka',
 'Suriname',
 'Sweden',
 'Switzerland',
 'Thailand',
 'Trinidad and Tobago',
 'Turkey',
 'Turkmenistan',
 'Ukraine',
 'United Arab Emirates',
 'United Kingdom',
 'United States',
 'Uruguay',
 'Uzbekistan

In [0]:
country_codes = pd.read_csv(path+"wikipedia-iso-country-codes.csv")

In [7]:
country_codes.head()

,English_short_name_lower_case,Alpha-2_code,Alpha-3_code,Numeric_code,ISO_3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS


'Cabo Verde',
 'Macau',
 'Republic of Korea',
 'Saint Vincent and Grenadines'}

In [0]:
d.country.values[d.country.values == "Cabo Verde"] = "Cape Verde"
d.country.values[d.country.values == "Macau"] = "Macao"
d.country.values[d.country.values == "Republic of Korea"] = "Korea, Republic of"
d.country.values[d.country.values == "Saint Vincent and Grenadines"] = "Saint Vincent and the Grenadines"

In [0]:
for i in range(d.values[:,0].shape[0]):
    new = country_codes.Numeric_code.values[country_codes.English_short_name_lower_case.values == d.country.values[i]]
    if (new.shape[0] != 0):
        d.country.values[i] = new[0]

In [0]:
set(d.values[:,0])

### Age
'5-14 years',
'15-24 years',
'25-34 years',
'35-54 years',
'55-74 years',
'75+ years'

In [0]:
d.age.values[d.age.values == "5-14 years"] = 0
d.age.values[d.age.values == "15-24 years"] = 1
d.age.values[d.age.values == "25-34 years"] = 2
d.age.values[d.age.values == "35-54 years"] = 3
d.age.values[d.age.values == "55-74 years"] = 4
d.age.values[d.age.values == "75+ years"] = 5

In [0]:
set(d.values[:,3])

{0, 1, 2, 3, 4, 5}

### Generation
'Boomers',
 'G.I. Generation',
 'Generation X',
 'Generation Z',
 'Millenials',
 'Silent'

In [0]:
d.generation.values[d.generation.values == "G.I. Generation"] = 0 #
d.generation.values[d.generation.values == "Silent"] = 1 #
d.generation.values[d.generation.values == "Boomers"] = 2 #
d.generation.values[d.generation.values == "Generation X"] = 3 #
d.generation.values[d.generation.values == "Millenials"] = 4 #
d.generation.values[d.generation.values == "Generation Z"] = 5 #

In [0]:
set(d.values[:,11])

{0, 1, 2, 3, 4, 5}

### Sex
'male', 'female'

In [0]:
d.sex.values[d.sex.values == "male"] = 0
d.sex.values[d.sex.values == "female"] = 1

### Gdp for year ($)

	63,067,077,179 = 63.07 ( se entiende como GDP ( Billions of US $) )

In [0]:
for i in range(d.values[:,0].shape[0]):
    res = d.gdp_for_year.values[i].split(',')
    d.gdp_for_year.values[i] = round(float(''.join(res))/1000000000,3)

In [0]:
 print(min(d.gdp_for_year.values))
 print(max(d.gdp_for_year.values))

0.047
18120.714


### NaN

La columna que mas datos presenta en NaN es **HDI_for_year** por tanto se propone reemplazar los valores NaN de cada pais con el primedio para no alterar la tendencia de cada pais.

In [26]:
NaN = []
for i in np.unique(d.country):
  mean = np.mean(d.HDI_for_year[d.country == i])
  if (np.isnan(mean)):
    NaN.append(i)
NaN

[132, 212, 296, 410, 446, 496, 533, 630, 643, 659, 674]

**Conclusion:** Existian paises a los cuales su columna de **HDI_for_year** era totalmente nula por tanto se procede a descartarlos del dataset dado que no hacen ningun aporte.

In [27]:
for i in NaN:
  d.drop(d.index[d.country == i].values, inplace=True)

d.reset_index(inplace=True)
d.drop(columns = "index", inplace=True)
d.index

RangeIndex(start=0, stop=26334, step=1)

Los paises que contienen registros sobre su **HDI_for_year** se procede a remplazar valores NaN con el promedio de los existentes.

In [28]:
for i in np.unique(d.country):
  d["HDI_for_year"][d.country==i]=d["HDI_for_year"][d.country==i].fillna(value=np.mean(d["HDI_for_year"][d.country==i]))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**NOTA:** Posiblemente al calcular el valor de la media se esta almacenando numeros de tipo flotante demaciado grandes por lo tanto corregir que solo guarde maximo 3 cifras decimales. Ejemplo: 0.7132000000000001

In [0]:
d.HDI_for_year = d.HDI_for_year.round(3)

**Prueba de nulidad** \\

In [30]:
np.prod(pd.notnull(d))

country              1
year                 1
sex                  1
age                  1
suicides_no          1
population           1
suicides/100k_pop    1
country-year         1
HDI_for_year         1
gdp_for_year         1
gdp_per_capita       1
generation           1
dtype: int64

### Zeros

Aquellos paises con numeros de suicidios **suidices_no** en 0 y años antes y despues no muy lejanos presentan valores con cierta tendencia es muy probable que ese registro no fuera tomado y/o se haya perdido por lo tanto lo mejor seria resplazarlos con la media.

In [0]:
def zeros(condicion, umbral):
  # Da informacion sobre que paises poseen una mayor cantidad de registros con numeros de suicidios en 0

  #umbral: porcentaje [0-100]
  #country_N: nombres de paises
  #country_No: codigo de paises
  #N_zeros: cantitad de registros que NO tienen 0
  #N_total: total de registros
  #NOTA: La columna "country" ya debe haber sido convertida a codigos

  cod_pais = []
  nom_pais = []
  no_zeros = []
  no_registros = []
  porcentaje = []

  for i in country_codes.Numeric_code.values:
    country_N = country_codes.English_short_name_lower_case[country_codes.Numeric_code == i].values[0]
    country_No = country_codes.Numeric_code[country_codes.Numeric_code == i].values[0]
    N_zeros = np.count_nonzero(d.suicides_no[d.country == i])
    N_total = len(d.suicides_no[d.country == i])
    if (N_total != 0):
      P_final = 100 - float(N_zeros)/N_total*100
      if (P_final > umbral and condicion == "mayor"): #Umbral de filtrado
        cod_pais.append(country_No)
        nom_pais.append(country_N)
        no_zeros.append(N_total - N_zeros)
        no_registros.append(N_total)
        porcentaje.append(P_final)
      if (P_final < umbral and condicion == "menor"): #Umbral de filtrado
        cod_pais.append(country_No)
        nom_pais.append(country_N)
        no_zeros.append(N_total - N_zeros)
        no_registros.append(N_total)
        porcentaje.append(P_final)

  zeros = {"cod_pais": cod_pais,
           "nom_pais": nom_pais,
           "no_zeros": no_zeros,
           "no_registros": no_registros,
           "porcentaje": porcentaje}
  zeros_df = pd.DataFrame(data=zeros, columns = ["cod_pais","nom_pais","no_zeros","no_registros","porcentaje"])
  return zeros_df

In [16]:
zeros("mayor", 99)

,cod_pais,nom_pais,no_zeros,no_registros,porcentaje
0,212,Dominica,12,12,100.0
1,659,Saint Kitts and Nevis,36,36,100.0


**Conclusion 1:** Tomando como referencia la cantidad de zeros presentes en la columna "suicides_no" con respecto a la cantidad de registros presentes y con un humbral del 100% se puede descartar los siguientes paises dado que sus datos no aparentar ser correctos o en su totalidad son faltantes:
- Dominica
- Saint Kitts and Nevis

In [0]:
d.drop(d.index[d.country == 212].values, inplace=True)
d.drop(d.index[d.country == 659].values, inplace=True)

d.reset_index(inplace=True)
d.drop(columns = "index", inplace=True)
d.index

RangeIndex(start=0, stop=26334, step=1)

In [0]:
d.head()

,country,year,sex,age,suicides_no,population,suicides/100k_pop,country-year,HDI_for_year,gdp_for_year,gdp_per_capita,generation
0,8,1987,0,1,21,312900,6.71,Albania1987,0.673,2.157,796,3
1,8,1987,0,3,16,308000,5.19,Albania1987,0.673,2.157,796,1
2,8,1987,1,1,14,289700,4.83,Albania1987,0.673,2.157,796,3
3,8,1987,0,5,1,21800,4.59,Albania1987,0.673,2.157,796,0
4,8,1987,0,2,9,274300,3.28,Albania1987,0.673,2.157,796,2


In [0]:
zeros("menor", 50)

**Conclusion 2:** Paises que cumplan un umbral del 50% y su poblacion no alcance la media de la poblacion general del data set tambien son posiblemente descartados dado que su significancia es minima.



In [31]:
z = zeros("menor", 30)#0.01
print(len(z))
print(len(np.unique(d.country)))
z

70
90


,cod_pais,nom_pais,no_zeros,no_registros,porcentaje
0,8,Albania,58,264,21.969697
1,32,Argentina,0,372,0.000000
2,51,Armenia,46,298,15.436242
3,36,Australia,3,360,0.833333
4,40,Austria,9,382,2.356021
5,31,Azerbaijan,27,192,14.062500
6,112,Belarus,0,252,0.000000
7,56,Belgium,7,372,1.881720
8,76,Brazil,0,372,0.000000
9,100,Bulgaria,8,360,2.222222


**Conclusion 3:** Paises que cumplan un umbral casi nulo (0%) son posibles candidatos a realizar analisis, dado que son aquellos que presentan mas de un 99.99% de datos sobre sus suicidios que ademas fueron diferentes de 0.

Dado el analisis anterior, ahora sabiendo los paises que presentan una mayor confianza respecto a los datos registrados sobre suicidios se pretende remplazar los zeros con la media respecto a cada pais.

In [32]:
for i in set(d.country.values):
  if i in z["cod_pais"].values:
    a = 0
  else:
    d.drop(d.index[d.country == i].values, inplace=True)
    
d.reset_index(inplace=True)
d.drop(columns = "index", inplace=True)
d.index

RangeIndex(start=0, stop=21698, step=1)

In [34]:
set(d["country"])

70

Ahora de acuerdo a cada pais que posea valores en 0 para su numero de suicidios se procede a remplazarlo con la media de **suicidios** de ese pais.

In [35]:
for i in np.unique(d.country):
  d["suicides_no"][d.country==i]=d["suicides_no"][d.country==i].replace(0, value=np.mean(d["suicides_no"][d.country==i]))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


**NOTA:** Posiblemente al calcular el valor de la media se esta almacenando numeros de tipo flotante demaciado grandes por lo tanto corregir que solo guarde 3 cifras decimales.

In [0]:
d.suicides_no = d.suicides_no.round(3)

## Exportar nuevo dataset:

In [37]:
#d_CO = d[d.country == 170]
#d_CO.drop(columns = "country", inplace=True)
#d_CO.drop(columns = "index", inplace=True)
#d_CO.reset_index(inplace=True)
#d_CO.index
#d_CO.head()
d.head()

,country,year,sex,age,suicides_no,population,suicides/100k_pop,country-year,HDI_for_year,gdp_for_year,gdp_per_capita,generation
0,8,1987,0,1,21.0,312900,6.71,Albania1987,0.673,2.157,796,3
1,8,1987,0,3,16.0,308000,5.19,Albania1987,0.673,2.157,796,1
2,8,1987,1,1,14.0,289700,4.83,Albania1987,0.673,2.157,796,3
3,8,1987,0,5,1.0,21800,4.59,Albania1987,0.673,2.157,796,0
4,8,1987,0,2,9.0,274300,3.28,Albania1987,0.673,2.157,796,2


In [38]:
d.shape

(21698, 12)

In [0]:
export_csv = d.to_csv (r'export_dataframe_70.csv', index = None, header=True)

<strike> **NOTA (19/07/14):** Revizar que el dataframe **export_dataframe.csv** aun tiene problemas de valores NaN. \\
**NOTA (19/07/14):** El problema parece ser de dos paises en concreto **Republic of Korea (410)** y **Russian Federation (643)** <strike>


